In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpa91235hz/pubring.gpg' created
gpg: /tmp/tmpa91235hz/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [3]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
import os
os.chdir("drive/Udacity/deep-learning/tv-script-generation")

In [5]:
!git status

On branch master
Your branch is up-to-date with 'origin/master'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   ../IMDB-keras/IMDB_In_Keras.ipynb
	modified:   ../IMDB-keras/IMDB_In_Keras_Solutions.ipynb
	modified:   ../batch-norm/Batch_Normalization_Exercises.ipynb
	modified:   ../batch-norm/Batch_Normalization_Lesson.ipynb
	modified:   ../batch-norm/Batch_Normalization_Solutions.ipynb
	deleted:    ../dcgan-svhn/DCGAN.ipynb
	deleted:    ../dcgan-svhn/DCGAN_Exercises.ipynb
	modified:   ../embeddings/Skip-Gram_word2vec.ipynb
	modified:   ../embeddings/Skip-Grams-Solution.ipynb
	modified:   ../face_generation/dlnd_face_generation.ipynb
	modified:   ../gan_mnist/Intro_to_GANs_Exercises.ipynb
	modified:   ../gan_mnist/Intro_to_GANs_Solution.ipynb
	modified:   ../image-classification/dlnd_image_classification.ipynb
	modified:   ../intro-to-rnns/Anna_KaRNN

# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/simpsons_script_lines_1.txt'
# data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
# text = text[81:]

ModuleNotFoundError: ignored

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [0]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 150701
Number of scenes: 1
Average number of sentences in each scene: 158270.0
Number of lines: 158271
Average number of words in each line: 11.663305343366757

The sentences 0 to 10:
"Miss Hoover: No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it."
Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?
Miss Hoover: I don't know. Although I'd sure like to talk to him. He didn't touch my lesson plan. What did he teach you?
Lisa Simpson: That life is worth living.
"Edna Krabappel-Flanders: The polls will be open from now until the end of recess. Now, (SOUR) just in case any of you have decided to put any thought into this, we'll have our final statements. Martin?"
Martin Prince: (HOARSE WHISPER) I don't think there's anything left to say.
Edna Krabappel-Flanders: Bart?
Bart Simpson: Victory party under the slide!
(Apartment Building: Ext.

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [0]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab_to_int = {word: ii for ii, word in enumerate(set(text))}
    int_to_vocab = {ii: word for ii, word in enumerate(set(text))}
#     counts = Counter(text)
#     vocab_to_int = dict()
#     int_to_vocab = dict()
#     for number, word in enumerate(counts):
#         vocab_to_int[word] = number
#         int_to_vocab[number] = word
    return vocab_to_int, int_to_vocab

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

ModuleNotFoundError: ignored

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [0]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    tokens = {
        ".":     "||Peroid||",
        ",":     "||Comma||",
        '"':     "||Quotation_Mark||",
        ";":     "||Semicolon||",
        "!":     "||Exclamation_Mark||",
        "?":     "||Question_Mark||",
        "(":     "||Left_Parentheses||",
        ")":     "||Right_Prentheses||",
        "--":     "||Dash||",
        "\n":    "||Return||"
    }
    return tokens

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

NameError: ignored

## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.0
Default GPU Device: /device:GPU:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [0]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input_ = tf.placeholder(tf.int32, [None, None], name="input")
    targets_ = tf.placeholder(tf.int32, [None, None], name="targets")
    learning_rate = tf.placeholder(tf.float32, name="learning_rate")
    return input_, targets_, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [0]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    n_layers = 2
    def make_cell(lstm_size):
        return tf.contrib.rnn.BasicLSTMCell(lstm_size)
    cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(n_layers)])
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32), name="initial_state")
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [0]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [0]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    output, state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    state = tf.identity(state, "final_state")
    return output, state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [0]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    output, final_state = build_rnn(cell, embed)
    
#     logits = tf.contrib.layers.fully_connected(output, vocab_size, activation_fn=None)
    logits = tf.contrib.layers.fully_connected(output,vocab_size,activation_fn=None, weights_initializer=tf.truncated_normal_initializer(stddev= 0.1),biases_initializer=tf.zeros_initializer())
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [0]:
from pprint import pprint
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
#     TODO: Implement Function
    words_per_batch = batch_size * seq_length
    n_batches = len(int_text)//words_per_batch
    
    int_text = int_text[:n_batches * words_per_batch]
    
    result = np.ndarray(shape=(n_batches,2,batch_size,seq_length), dtype=np.int32)
    skipdistance = n_batches*seq_length
    
    for i in range(n_batches):
        batch_idx = i*seq_length
        x , y= [], []
        for bb in range(batch_size):
            idx = batch_idx + (bb*skipdistance) # get starting index for batch
            x_idx = idx
            y_idx = idx+1          
            result[i][0][bb] = int_text[x_idx:x_idx+seq_length]
            result[i][1][bb] = int_text[y_idx:y_idx+seq_length] if len(int_text[y_idx:y_idx+seq_length])==seq_length else int_text[y_idx:y_idx+seq_length] + int_text[0:1]
            
    return result

#     n_batches = int(len(int_text) / (batch_size * seq_length))

#     #Drop the last few characters to make only full batches
#     xdata = np.array(int_text[: n_batches * batch_size * seq_length])
#     ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])

#     x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
#     y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)

#     return np.array(list(zip(x_batches, y_batches)))

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [0]:
# Number of Epochs
num_epochs = 20
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 1024
# Embedding Dimension Size
embed_dim = 500
# Sequence Length
seq_length = 12
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 50

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/885   train_loss = 10.925
Epoch   0 Batch   50/885   train_loss = 5.155
Epoch   0 Batch  100/885   train_loss = 4.664
Epoch   0 Batch  150/885   train_loss = 4.526
Epoch   0 Batch  200/885   train_loss = 4.372
Epoch   0 Batch  250/885   train_loss = 4.440
Epoch   0 Batch  300/885   train_loss = 4.090
Epoch   0 Batch  350/885   train_loss = 4.254
Epoch   0 Batch  400/885   train_loss = 4.283
Epoch   0 Batch  450/885   train_loss = 4.250
Epoch   0 Batch  500/885   train_loss = 4.161
Epoch   0 Batch  550/885   train_loss = 4.177
Epoch   0 Batch  600/885   train_loss = 4.031
Epoch   0 Batch  650/885   train_loss = 4.170
Epoch   0 Batch  700/885   train_loss = 4.253
Epoch   0 Batch  750/885   train_loss = 4.073
Epoch   0 Batch  800/885   train_loss = 4.059
Epoch   0 Batch  850/885   train_loss = 4.114
Epoch   1 Batch   15/885   train_loss = 3.913
Epoch   1 Batch   65/885   train_loss = 3.906
Epoch   1 Batch  115/885   train_loss = 4.077
Epoch   1 Batch  165/885   train_

Epoch   3 Batch  595/885   train_loss = 3.542
Epoch   3 Batch  645/885   train_loss = 3.585
Epoch   3 Batch  695/885   train_loss = 3.577
Epoch   3 Batch  745/885   train_loss = 3.613
Epoch   3 Batch  795/885   train_loss = 3.419
Epoch   3 Batch  845/885   train_loss = 3.507
Epoch   4 Batch   10/885   train_loss = 3.539
Epoch   4 Batch   60/885   train_loss = 3.532
Epoch   4 Batch  110/885   train_loss = 3.403
Epoch   4 Batch  160/885   train_loss = 3.420
Epoch   4 Batch  210/885   train_loss = 3.375
Epoch   4 Batch  260/885   train_loss = 3.527
Epoch   4 Batch  310/885   train_loss = 3.486
Epoch   4 Batch  360/885   train_loss = 3.437
Epoch   4 Batch  410/885   train_loss = 3.429
Epoch   4 Batch  460/885   train_loss = 3.425
Epoch   4 Batch  510/885   train_loss = 3.424
Epoch   4 Batch  560/885   train_loss = 3.340
Epoch   4 Batch  610/885   train_loss = 3.374
Epoch   4 Batch  660/885   train_loss = 3.409
Epoch   4 Batch  710/885   train_loss = 3.292
Epoch   4 Batch  760/885   train_l

Epoch   7 Batch  305/885   train_loss = 3.256
Epoch   7 Batch  355/885   train_loss = 3.268
Epoch   7 Batch  405/885   train_loss = 3.253
Epoch   7 Batch  455/885   train_loss = 3.218
Epoch   7 Batch  505/885   train_loss = 3.346
Epoch   7 Batch  555/885   train_loss = 3.380
Epoch   7 Batch  605/885   train_loss = 3.340
Epoch   7 Batch  655/885   train_loss = 3.274
Epoch   7 Batch  705/885   train_loss = 3.248
Epoch   7 Batch  755/885   train_loss = 3.332
Epoch   7 Batch  805/885   train_loss = 3.181
Epoch   7 Batch  855/885   train_loss = 3.161
Epoch   8 Batch   20/885   train_loss = 3.160
Epoch   8 Batch   70/885   train_loss = 3.425
Epoch   8 Batch  120/885   train_loss = 3.242
Epoch   8 Batch  170/885   train_loss = 3.205
Epoch   8 Batch  220/885   train_loss = 3.262
Epoch   8 Batch  270/885   train_loss = 3.254
Epoch   8 Batch  320/885   train_loss = 3.248
Epoch   8 Batch  370/885   train_loss = 3.277
Epoch   8 Batch  420/885   train_loss = 3.346
Epoch   8 Batch  470/885   train_l

Epoch  11 Batch   15/885   train_loss = 3.047
Epoch  11 Batch   65/885   train_loss = 3.176
Epoch  11 Batch  115/885   train_loss = 3.260
Epoch  11 Batch  165/885   train_loss = 3.091
Epoch  11 Batch  215/885   train_loss = 3.108
Epoch  11 Batch  265/885   train_loss = 3.092
Epoch  11 Batch  315/885   train_loss = 3.136
Epoch  11 Batch  365/885   train_loss = 3.096
Epoch  11 Batch  415/885   train_loss = 3.221
Epoch  11 Batch  465/885   train_loss = 3.201
Epoch  11 Batch  515/885   train_loss = 3.110
Epoch  11 Batch  565/885   train_loss = 3.122
Epoch  11 Batch  615/885   train_loss = 3.097
Epoch  11 Batch  665/885   train_loss = 3.119
Epoch  11 Batch  715/885   train_loss = 3.016
Epoch  11 Batch  765/885   train_loss = 3.134
Epoch  11 Batch  815/885   train_loss = 3.146
Epoch  11 Batch  865/885   train_loss = 3.160
Epoch  12 Batch   30/885   train_loss = 3.192
Epoch  12 Batch   80/885   train_loss = 3.212
Epoch  12 Batch  130/885   train_loss = 3.063
Epoch  12 Batch  180/885   train_l

Epoch  14 Batch  610/885   train_loss = 3.077
Epoch  14 Batch  660/885   train_loss = 3.091
Epoch  14 Batch  710/885   train_loss = 2.956
Epoch  14 Batch  760/885   train_loss = 3.079
Epoch  14 Batch  810/885   train_loss = 2.975
Epoch  14 Batch  860/885   train_loss = 3.035
Epoch  15 Batch   25/885   train_loss = 3.075
Epoch  15 Batch   75/885   train_loss = 3.079
Epoch  15 Batch  125/885   train_loss = 3.088
Epoch  15 Batch  175/885   train_loss = 3.082
Epoch  15 Batch  225/885   train_loss = 3.007
Epoch  15 Batch  275/885   train_loss = 3.079
Epoch  15 Batch  325/885   train_loss = 3.097
Epoch  15 Batch  375/885   train_loss = 3.076
Epoch  15 Batch  425/885   train_loss = 3.136
Epoch  15 Batch  475/885   train_loss = 3.158
Epoch  15 Batch  525/885   train_loss = 3.136
Epoch  15 Batch  575/885   train_loss = 3.072
Epoch  15 Batch  625/885   train_loss = 3.035
Epoch  15 Batch  675/885   train_loss = 3.147
Epoch  15 Batch  725/885   train_loss = 2.954
Epoch  15 Batch  775/885   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [0]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

print(vocab_to_int['homer'])

25939


## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [0]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [0]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    return int_to_vocab[np.random.choice(len(int_to_vocab), 1, p=probabilities)[0]]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [0]:
gen_length = 400
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'homer'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save


INFO:tensorflow:Restoring parameters from ./save


homer: hey, him, mister goblin likes mindy and i walk closer for starters an elephant. for the most movies like something kinda causing no choice?
apu nahasapeemapetilon: the new al mill.
" drederick tatum: good morning, steady milk."
rev. timothy lovejoy:(indignant) hello, mr. simpson, this is just bound to make you want me another reputation. and i'll go to it, but in order of beautiful?"
(springfield town hall: ext. town library - night)
(bangalore airport floor: int. lunchroom - day)
(springfield elementary school: ext. springfield elementary - picnic - establishing)
" jimbo jones: orange drink: please don't smithers. how'd you please buy it?"
" homer simpson: hazel. simpson, marked five, etc.)"
" dr. julius hibbert: finally. what's wrong with charlton baldness?
bart simpson: not so cute. i mean, thank you methasorbizone tartrate""."
ken: would someone increase your language?!
curator:(climbing noise)
" moe szyslak: acres of orleans, ba-ba-ba-ba-ba..."
homer simpson:(angry grumbles

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.